In [2]:
!pip install torch transformers langchain sentence-transformers pinecone-client pypdf langchain_community langchain_huggingface chromadb transformers torch sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.2 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
import torch
import logging
from typing import List, Optional
from dataclasses import dataclass

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class Config:
    """Configuration settings for the QA system"""
    pdf_path: str
    persist_directory: str = "./chroma_db"
    chunk_size: int = 1000  # Increased chunk size for more context
    chunk_overlap: int = 200  # Increased overlap
    model_id: str = "google/flan-t5-large"
    embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2"
    max_length: int = 512  # Increased max_length for longer responses
    top_k: int = 4


class PDFQuestionAnswering:
    def __init__(self, config: Config):
        self.config = config
        self.qa_chain = None

        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.config.embedding_model,
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
        )

        # Initialize text splitter with adjusted chunk size
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config.chunk_size,
            chunk_overlap=self.config.chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )


    def process_pdf(self) -> List:
        """Process PDF document and split into chunks"""
        try:
            logger.info(f"Processing PDF: {self.config.pdf_path}")
            loader = PyPDFLoader(self.config.pdf_path)
            documents = loader.load()
            splits = self.text_splitter.split_documents(documents)
            logger.info(f"Created {len(splits)} text chunks")
            return splits
        except Exception as e:
            logger.error(f"Error processing PDF: {str(e)}")
            raise

    def initialize_qa_system(self):
        """Initialize the QA system with the PDF content"""
        try:
            # Process the PDF
            splits = self.process_pdf()

            # Create vector store
            vectorstore = Chroma.from_documents(
                documents=splits,
                embedding=self.embeddings,
                persist_directory=self.config.persist_directory
            )

            # Initialize language model with longer responses
            tokenizer = AutoTokenizer.from_pretrained(self.config.model_id)
            model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_id)

            pipe = pipeline(
                "text2text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=self.config.max_length,
                model_kwargs={
                    "temperature": 0.3,
                    "num_beams": 6,  # Increased beams
                    "no_repeat_ngram_size": 3
                },
                device=0 if torch.cuda.is_available() else -1
            )

            llm = HuggingFacePipeline(pipeline=pipe)

            # Adjusted prompt for more detailed answers
            template = """Use the following pieces of context to answer the question at the end. Please provide a detailed, in-depth response. Include examples or elaborate on the concepts wherever possible. If you don't know the answer, just say you don't know. Do not make up an answer.
            Context: {context}
            Question: {question}
            Answer: """

            prompt = PromptTemplate(
                template=template,
                input_variables=["context", "question"]
            )

            # Create QA chain
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=vectorstore.as_retriever(
                    search_kwargs={'k': self.config.top_k}
                ),
                chain_type_kwargs={"prompt": prompt},
                return_source_documents=True
            )

            logger.info("QA system initialized successfully")

        except Exception as e:
            logger.error(f"Error initializing QA system: {str(e)}")
            raise


    def ask_question(self, question: str) -> Optional[str]:
        """Process a question and return the answer"""
        try:
            if not self.qa_chain:
                raise ValueError("QA system not initialized")
            result = self.qa_chain.invoke({"query": question})
            return result['result']
        except Exception as e:
            logger.error(f"Error processing question: {str(e)}")
            return f"Sorry, I encountered an error: {str(e)}"

# Upload PDF and initialize the system
from google.colab import files
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]

# Create config and initialize system
config = Config(
    pdf_path=f"/content/{pdf_filename}",
    persist_directory="/content/chroma_db"
)

# Initialize QA system
qa_system = PDFQuestionAnswering(config)
qa_system.initialize_qa_system()

# Interactive session
def interactive_qa():
    print("\nSystem is ready! You can now ask questions about the PDF content.")
    while True:
        question = input("\nAsk a question (or type 'exit' to quit): ")
        if question.lower() == 'exit':
            print("Goodbye!")
            break

        answer = qa_system.ask_question(question)
        print("\nAnswer:", answer)

# Start interactive session
interactive_qa()


Saving formula_one_tutorial.pdf to formula_one_tutorial.pdf


Device set to use cpu



System is ready! You can now ask questions about the PDF content.

Ask a question (or type 'exit' to quit): 1. What is Formula One?


Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors



Answer: 1. an international auto racing sport

Ask a question (or type 'exit' to quit): Explain the objective of a Formula One race.

Answer: The objective of a Formula 1 contest is to determine the winner of a race. The driver who crosses the finish line first after completing a pre-determined number of laps is declared the winner.

Ask a question (or type 'exit' to quit): What is the origin of the term 'Formula' in Formula One

Answer: The name ‘Formula’ comes from the set of rules that the participating cars and drivers must follow.

Ask a question (or type 'exit' to quit): 12. What types of tires are used in Formula One, and how do they impact racing?

Answer: Smooth thread, slick tires.

Ask a question (or type 'exit' to quit): Explain the format of a Formula One Grand Prix weekend.

Answer: Different events take place on each of these 3 days. They are -  Friday - Free Practice Sessions  Saturday - Free Practice Session plus Qualifying Session  Sunday - Race Day Practice Sessions